In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.concat([pd.read_csv('../data/On_Time_On_Time_Performance_2016_1.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_2.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_3.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_4.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_5.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_6.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_7.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_8.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_9.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_10.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_11.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_12.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2017_1.csv')])

/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (48,77,84,85) have mixed types. Specify dty

# Day of the week, Day of the month, month, UniqueCarrier, OriginAirportID, DestAirportID, CRSDepTime 

In [4]:
float(df[df['DepDelay'] > 0].shape[0])/df.shape[0]

0.3444744486149967

In [5]:
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2016,1,1,6,3,2016-01-06,AA,19805,AA,N4YBAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,1,1,7,4,2016-01-07,AA,19805,AA,N434AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,1,1,8,5,2016-01-08,AA,19805,AA,N541AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,1,1,9,6,2016-01-09,AA,19805,AA,N489AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,1,1,10,7,2016-01-10,AA,19805,AA,N439AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.shape

(6067675, 110)

In [7]:
df = df[df['Cancelled']<1] # The flight is cancelled

In [8]:
df.shape

(5992928, 110)

In [9]:
df = df[df['Diverted']<1] # The flight is diverted

In [10]:
df.shape

(5977790, 110)

In [11]:
###### Create a field for logistic regression

df['Class'] = 'No Delay'
df['Class'][(df['DepDelay'] >= 15) & (df['DepDelay'] < 30)] = 'Minor Delay'
df['Class'][(df['DepDelay'] >= 30) & (df['DepDelay'] < 60)] = 'Delay'
df['Class'][(df['DepDelay'] >= 60) & (df['DepDelay'] < 120)] = 'Significant Delay'
df['Class'][df['DepDelay'] > 120] = 'Severe Delay'


/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/_

In [12]:
df.groupby(['Class']).count()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
Class,,,,,,,,,,,,,,,,,,,,,
Delay,315208,315208,315208,315208,315208,315208,315208,315208,315208,315208,...,0,0,0,0,0,0,0,0,0,0
Minor Delay,382848,382848,382848,382848,382848,382848,382848,382848,382848,382848,...,0,0,0,0,0,0,0,0,0,0
No Delay,4939438,4939438,4939438,4939438,4939438,4939438,4939438,4939438,4939438,4939438,...,0,0,0,0,0,0,0,0,0,0
Severe Delay,126865,126865,126865,126865,126865,126865,126865,126865,126865,126865,...,0,0,0,0,0,0,0,0,0,0
Significant Delay,213431,213431,213431,213431,213431,213431,213431,213431,213431,213431,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# std_pop = df['DepDelay'].mean() + 3*df['DepDelay'].std() # Need to normalize the flight delay time. removing outliers.

In [14]:
# df.set_value(df[df['DepDelay'] < - std_pop].index.values,'DepDelay',- std_pop)

In [15]:
# df.set_value(df[df['DepDelay'] > std_pop].index.values,'DepDelay',std_pop)

In [16]:
# df['DepDelay'].max()

In [17]:
# df['DepDelay'].hist()

In [18]:
df.shape

(5977790, 111)

In [71]:
df_sub = df[['DayOfWeek', 'DayofMonth', 'Month', 'AirlineID', 'OriginAirportID', 'DestAirportID', 'CRSDepTime', 'Class']]

In [72]:
df_sub = df_sub.dropna()

In [73]:
df_sub.dtypes

DayOfWeek           int64
DayofMonth          int64
Month               int64
AirlineID           int64
OriginAirportID     int64
DestAirportID       int64
CRSDepTime          int64
Class              object
dtype: object

In [74]:
df_sub.shape

(5977790, 8)

In [75]:
df_sub.DayOfWeek=df_sub.DayOfWeek.astype(str)

In [76]:
df_sub.DayofMonth=df_sub.DayofMonth.astype(str)

In [77]:
df_sub.Month=df_sub.Month.astype(str)

In [78]:
df_sub.OriginAirportID=df_sub.OriginAirportID.astype(str)

In [79]:
df_sub.DestAirportID=df_sub.DestAirportID.astype(str)

In [81]:
df_sub.AirlineID=df_sub.AirlineID.astype(str)

In [82]:
df_sub.dtypes

DayOfWeek          object
DayofMonth         object
Month              object
AirlineID          object
OriginAirportID    object
DestAirportID      object
CRSDepTime          int64
Class              object
dtype: object

In [29]:
# df_sub['DepDelay'].hist()

In [30]:
# df_sub['DepDelay'].median()

In [31]:
# df_sub.set_value(df_sub[df_sub['DepDelay'] < 0].index.values,'DepDelay',0.001)

In [32]:
# df_sub.set_value(df_sub[df_sub['DepDelay'] == 0].index.values,'DepDelay',0.001)

In [33]:
# df_sub['DepDelay'].hist()

In [34]:
# import math
# import numpy as np

In [35]:
# df_sub['DepDelayLog'] = df_sub['DepDelay'].apply(np.log)

In [36]:
# df_sub['DepDelay'].hist()

In [37]:
# from sklearn import preprocessing

In [38]:
# ss = preprocessing.StandardScaler()

In [39]:
# ss = ss.fit(df_sub['DepDelay'])

In [40]:
# ss.transform(df_sub['DepDelay']).mean()

In [41]:
# df_sub['DepDelayNorm'] = ss.transform(df_sub['DepDelay'])

In [42]:
# df_sub['DepDelayNorm'].hist()

In [83]:
df_sub['CRSDepHH'] = df_sub.CRSDepTime/100

In [84]:
df_sub['CRSDepHH'] = df_sub['CRSDepHH'].astype(int)

In [85]:
df_sub['CRSDepHH'].tail()

450012     8
450013    14
450014    14
450015     7
450016    16
Name: CRSDepHH, dtype: int64

In [86]:
df_sub.CRSDepTime.tail()

450012     800
450013    1430
450014    1415
450015     755
450016    1620
Name: CRSDepTime, dtype: int64

In [87]:
df_sub.sample(10)

,DayOfWeek,DayofMonth,Month,AirlineID,OriginAirportID,DestAirportID,CRSDepTime,Class,CRSDepHH
314782,6,20,8,19805,11057,11697,1625,No Delay,16
87329,2,29,11,19790,10397,15304,1330,No Delay,13
324136,3,23,3,19393,12892,12889,1630,No Delay,16
315429,2,10,1,19393,13796,13891,635,No Delay,6
328771,6,27,8,19805,14843,13303,600,No Delay,6
117665,2,6,9,19790,10397,14986,1026,Significant Delay,10
389443,3,30,11,19805,14100,14307,945,Minor Delay,9
11757,4,22,9,19805,12892,10397,2120,No Delay,21
437095,2,29,3,19977,13930,14107,830,No Delay,8
440727,3,14,12,20366,12197,10397,718,Significant Delay,7


In [88]:
# from sklearn import linear_model

In [89]:
# reg = linear_model.Ridge (alpha = .5)

In [90]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()

In [91]:
x, y = df_sub[['DayOfWeek', 
                'DayofMonth', 
               'Month', 
               'AirlineID', 
               'OriginAirportID', 
               'DestAirportID' ,
                'CRSDepHH']], df_sub['Class']
               

In [92]:
x.dtypes

DayOfWeek          object
DayofMonth         object
Month              object
AirlineID          object
OriginAirportID    object
DestAirportID      object
CRSDepHH            int64
dtype: object

In [93]:
x.sample()

,DayOfWeek,DayofMonth,Month,AirlineID,OriginAirportID,DestAirportID,CRSDepHH
97290,3,21,12,20409,12892,11697,12


In [94]:
y.dtypes

dtype('O')

In [95]:
from sklearn.model_selection import train_test_split

In [96]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [97]:
#X_train = X_train.reshape(-1,1)

In [98]:
X_train.shape

(4184453, 7)

In [99]:
#y_train = y_train.reshape(-1,1)

In [100]:
y_train.shape

(4184453,)

In [101]:
# reg = reg.fit(X_train,y_train)

In [102]:
# Decision Tree fitting a model
clf = clf.fit(X_train, y_train)

In [103]:
from sklearn.metrics import precision_recall_fscore_support

In [104]:
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)


In [105]:
#Test
precision_recall_fscore_support(y_test, y_pred, average='macro')

(0.25679236566064112, 0.26445492994105341, 0.26006701042424385, None)

In [106]:
#Train
precision_recall_fscore_support(y_train, y_pred_train, average='macro')

(0.99586987879137356, 0.99612815454168402, 0.9959845617468126, None)

In [107]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.70824892365461711

In [109]:
accuracy_score(y_train, y_pred_train)

0.99850087932640175

In [111]:
from sklearn.externals import joblib
filename = 'final_model.pkl'
joblib.dump(clf, filename)



['final_model.pkl']

In [112]:
# 1 1 1 19805 11298 11433 11
day_of_week = str(1)
day_of_month = str(1)
month = str(1)
carrier = str(19805)
origin = str(11298)
destination = str(11433)
departure_time = str(11)